<a href="https://colab.research.google.com/github/qwiksilva/cs224w-github-rec/blob/master/gharchive_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
%load_ext google.colab.data_table
# %unload_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [0]:
%%bigquery --project cs325b-218620 issue_comment_df
SELECT
  JSON_EXTRACT_SCALAR(payload, '$.comment.id') as id,
  JSON_EXTRACT_SCALAR(payload, '$.comment.body') as comment,
  JSON_EXTRACT_SCALAR(payload, '$.comment.user.id') as commenter_id,
  JSON_EXTRACT_SCALAR(payload, '$.comment.user.login') as commenter_login,
  #JSON_EXTRACT_SCALAR(payload, '$.issue.id') as pr_id,
  JSON_EXTRACT_SCALAR(payload, '$.issue.number') as number,
  JSON_EXTRACT_SCALAR(payload, '$.issue.user.id') as user_id,
  JSON_EXTRACT_SCALAR(payload, '$.issue.user.login') as username,
  JSON_EXTRACT_SCALAR(payload, '$.issue.title') as title,
  JSON_EXTRACT_SCALAR(payload, '$.issue.body') as description,
  JSON_EXTRACT_SCALAR(payload, '$.issue.created_at') as pr_created_at,
  JSON_EXTRACT_SCALAR(payload, '$.comment.created_at') as comment_created_at
FROM `githubarchive.year.20*`
WHERE
  _TABLE_SUFFIX BETWEEN '15' AND '18'
  AND type = 'IssueCommentEvent'
  AND repo.name = "kubernetes/kubernetes"
  AND JSON_EXTRACT_SCALAR(payload, '$.action')="created"
  AND JSON_EXTRACT_SCALAR(payload, '$.issue.number') IN 
    (SELECT  DISTINCT
    JSON_EXTRACT_SCALAR(payload, '$.pull_request.number') as number
    FROM `githubarchive.year.20*`
    where 
    _TABLE_SUFFIX BETWEEN '15' AND '18'
    AND type = 'PullRequestEvent'
    AND repo.name = "kubernetes/kubernetes"
    )

In [0]:
%%bigquery --project cs325b-218620 pr_comment_df

SELECT
  JSON_EXTRACT_SCALAR(payload, '$.comment.id') as id,
  JSON_EXTRACT_SCALAR(payload, '$.comment.body') as comment,
  JSON_EXTRACT_SCALAR(payload, '$.comment.user.id') as commenter_id,
  JSON_EXTRACT_SCALAR(payload, '$.comment.user.login') as commenter_login,
  #JSON_EXTRACT_SCALAR(payload, '$.pull_request.id') as pr_id,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.number') as number,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.id') as user_id,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.user.login') as username,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.title') as title,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.body') as description,
  JSON_EXTRACT_SCALAR(payload, '$.pull_request.created_at') as pr_created_at,
  JSON_EXTRACT_SCALAR(payload, '$.comment.created_at') as comment_created_at
FROM `githubarchive.year.20*`

WHERE
  _TABLE_SUFFIX BETWEEN '15' AND '18'
  AND type='PullRequestReviewCommentEvent'
  AND repo.name = "kubernetes/kubernetes"
  AND JSON_EXTRACT_SCALAR(payload, '$.action')="created"

In [0]:
comment_df = pr_comment_df.append(issue_comment_df)
comment_df.to_csv('kubernetes.csv')

In [21]:
len(comment_df.number.unique())

35452

In [13]:
comment_df

,id,comment,commenter_id,commenter_login,number,user_id,username,title,description,pr_created_at,comment_created_at
0,83223424,Whitespace removed,13948445,JacobTanenbaum,27206,13948445,JacobTanenbaum,Add 'kubectl set resources',"Add ""kubectl set resources"" for easier updatin...",2016-06-10T22:17:36Z,2016-10-13T14:04:54Z
1,71956732,It is for sorting. See first example here http...,11994812,mwielgus,29455,11994812,mwielgus,Replica location planner for Federated Replica...,Requires #29385 to be merged.\r\n\r\ncc: @quin...,2016-07-22T15:10:49Z,2016-07-22T22:52:15Z
2,79633255,"yeah, we could integrate dumping into the bina...",826111,jayunit100,32902,826111,jayunit100,Viper direct bindings to TestContext struct wi...,Part of #31453 to support hierarchichal parame...,2016-09-16T21:00:39Z,2016-09-20T15:04:17Z
3,92006105,Once both annotation conversations are in that...,87555,rrati,37299,87555,rrati,[scheduling] Moved node affinity from annotati...,Converted node affinity from annotations to ap...,2016-11-22T18:32:03Z,2016-12-12T18:17:54Z
4,66541531,Gone now with sets.String,13481082,therc,26900,13481082,therc,AWS: cache values from getInstancesByNodeName(),Fixes #25610 by reducing redundant calls to De...,2016-06-06T17:54:03Z,2016-06-09T23:47:07Z
...,...,...,...,...,...,...,...,...,...,...,...
705206,326383169,/retest\nThis bot automatically retries jobs t...,30488982,fejta-bot,51519,13374016,hzxuzhonghu,update deprecated interface and fix bug not re...,<!-- Thanks for sending a pull request! Here...,2017-08-29T12:19:56Z,2017-08-31T18:30:51Z
705207,326841400,/retest\nThis bot automatically retries jobs t...,30488982,fejta-bot,51868,730123,sttts,client-go: fix 'go build ./...',Test-only directories seem to confuse go-build...,2017-09-03T13:16:36Z,2017-09-04T00:12:21Z
705208,331331239,/retest\nThis bot automatically retries jobs t...,30488982,fejta-bot,52322,5459214,davidz627,Fixes issue where PVCs using `standard` Storag...,Fixes #50115\r\n\r\nChanged GetAllZones to onl...,2017-09-12T03:23:27Z,2017-09-22T02:12:24Z
705209,350207355,/retest\nThis bot automatically retries jobs t...,30488982,fejta-bot,55832,8268008,ceridwen,Remove deprecated API definitions to make the ...,<!-- Thanks for sending a pull request! Here...,2017-11-16T01:32:16Z,2017-12-08T08:45:49Z


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [19]:
uploaded = drive.CreateFile({'title': 'kubernetes.csv'})
uploaded.SetContentFile('kubernetes.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1_7UJg1gfcot7-FWoFlMm7bjvX7lutq7v
